# Responsável por tratar os dados pertinente a cliente

## Importando bibliotecas

In [11]:
import sys, os

# Carregando pacote que estao fora de /transform
path_raiz = os.path.abspath(os.path.join("..", ".."))
sys.path.append(path_raiz)

In [12]:
import pandas as pd
from src.utils import carregar_tabela_excel
import json

## Importando e tratando os dados

In [13]:
source_input = "../../data/baseTesteIXC.xlsx"
source_output = "../../data/baseTesteIXC_Transform.xlsx"
df = carregar_tabela_excel(source_input)

### Tratando CPF_CNPJ e Removendo Duplicados

In [14]:
# Garante que a coluna seja interpretada como string
df["cpf_cnpj"] = df["cpf_cnpj"].astype(str)

# Remove os caracteres '.', '-', '/' usando expressão regular
df["cpf_cnpj"] = df["cpf_cnpj"].str.replace(r"[./-]", "", regex=True)

# Remove duplicados do CPF tratando como um unico cliente
df = df.drop_duplicates(subset=["cpf_cnpj"], keep="first")

### Tratando numero de Celular

In [15]:
# Converte a coluna Celular em string
df["Celular"] = df["Celular"].astype(str)

# Remove os caracteres '(', ')' e '-' usando expressão regular
df["Celular"] = df["Celular"].str.replace(r"[()\-]", "", regex=True)

# Substitui valores 'nan' ou string vazia por '99999999999'
df["Celular"] = df["Celular"].replace(["nan", ""], "99999999999")

### Tratando rg_ie

In [16]:
## Substitui NaN por vazio no campo rg_ie
df["rg_ie"] = df["rg_ie"].fillna("")

### Tratando Complemento e Número 

In [17]:
## Substitui NaN por vazio no campo Complemento
df["Complemento"] = df["Complemento"].fillna("")


## Tratando número da casa
import re

def tratar_numero(valor):
    # Converte para string
    texto = str(valor)

    # Captura todos os dígitos (0-9) presentes no texto
    digits = re.findall(r"\d+", texto)

    if len(digits) > 0:
        # Se encontrar dígitos, junta tudo numa única string
        return "".join(digits)
    else:
        # Se não houver dígitos, retorna "SN"
        return "SN"

# Aplica a função na coluna Numero
df["Numero"] = df["Numero"].apply(tratar_numero)
df

,id,razaoSocial,Email,Celular,cpf_cnpj,rg_ie,filial_id,idCidade,Endereco,Complemento,Numero,Bairro,Cep,idPlanoIXC,idVencimentoIXC,Login,Senha
0,1306,Abelanio Veríssimo Peixoto,teste@gmail.com,99999999999,01727564057,Isento,35,1659,Vila São Sebastião,Próximo ao cacimbão,0,Centro,57830-000,NaN,NaN,teste1@radios.com,peixoto
1,897,Aclecia da Silva,teste@gmail.com,99999999999,27010089051,30234875,35,1659,Maria de Lurdes,Próximo ao mirante,SN,Centro,57830-000,NaN,NaN,teste2@radios.com,silva
2,1120,Adeilton da Mata,teste@gmail.com,99999999999,71988249007,40259595,35,1659,Ass. Nova Esperança,próximo a casa de farinha,SN,Zona Rural,57830-000,NaN,NaN,teste3@radios.com,mata
3,1524,Adeilton da Silva,teste@gmail.com,99999999999,36544973068,3614522-0,35,1659,Gravatá,Lote do Sr. Del,SN,Zona Rural,57830-000,NaN,NaN,teste4@radios.com,silva
4,1427,Adeilton Teixeira de Lima,teste@gmail.com,99999999999,62950454054,36145220,35,1659,João Lyra / Qd: A,,24,Conj. Raimundo Nonato Lopes,57830-000,NaN,NaN,teste5@radios.com,lima
5,503,Adelmo Luiz Berto De Oliveira,teste@gmail.com,99999999999,54098736020,ISENTO,35,1659,Rua: Manoel Luiz da Silva,Próximo ao Mirante,SN,Centro,57830-000,NaN,NaN,teste6@radios.com,silva
6,1288,Ademar Manoel da Silva,teste@gmail.com,99999999999,85604945000175,30972841,35,1659,Ass.: Flor do Mundaú,,SN,Zona Rural,57830-000,NaN,NaN,teste7@radios.com,silva
7,565,Adilson Pereira Da Silva,teste@gmail.com,99999999999,28043105000147,35170212,35,1659,Ass.: Flor do Mundaú,Pop Matinha,SN,Zona Rural,57830-000,NaN,NaN,teste8@radios.com,silva
8,1057,Adriana Paulino da Silva,teste@gmail.com,99999999999,92678107000101,ISENTO,35,1659,Platô 1/ Q: 02,,06,Conj. Raimundo Nonato Lopes,57830-000,NaN,NaN,teste9@radios.com,silva


### Exportando

In [18]:
# Converter colunas que devem preservar zeros à esquerda para string
colunas_texto = ['cpf_cnpj', 'Cep', 'Numero']
for coluna in colunas_texto:
    df[coluna] = df[coluna].astype(str).str.strip()  # Assegura que sejam strings e remove espaços

df.to_excel(source_output)